In [61]:
import pandas as pd
from sklearn.cluster import KMeans


def make_full_filled_clusters(temp_df, drive_no, min_req, max_req):
    count_sum = temp_df['count'].sum()

    if count_sum < max_req:
        if count_sum < 0.2 * min_req:
            temp_df['drive_no'] = drive_no if drive_no%10 == 0 else drive_no-1
            return temp_df, drive_no
        temp_df['drive_no'] = drive_no
        drive_no += 1
        return temp_df, drive_no
    
    return_df = pd.DataFrame()

    kmeans = KMeans(n_clusters=2).fit(temp_df[['lat','lng']])
    temp_df['internal_clusters'] = list(kmeans.labels_)
    for i in range(2):
        df = temp_df[temp_df['internal_clusters']==i]
        res, drive_no = make_full_filled_clusters(df, drive_no, min_req, max_req)
        return_df = pd.concat([return_df, res])
    return return_df, drive_no

BASEPATH = '/home/jasmeet16-jtg/projects/calyx_ds/src/models/drive_recommendation'
min_req, max_req = 200, 400
cluster_on_distance_file_path = os.path.join(BASEPATH, 'pool_recommendation_data/demo_drive_recommendation_cluster_on_distance.csv')
output_file_path = os.path.join(BASEPATH, 'pool_recommendation_data/test.csv')
cluster_on_distance = pd.read_csv(cluster_on_distance_file_path)
fullfilled_drives = pd.DataFrame()
for i in range(cluster_on_distance['cluster_number'].nunique()):
    temp_df = cluster_on_distance[cluster_on_distance['cluster_number']==i]
    group_obj = temp_df.groupby(by=['lat', 'lng'])['count'].sum().reset_index().rename(columns={'count':'count_sum'})
    drive_no = i*10 + 0
    for index, row in group_obj.iterrows():
        if row['count_sum'] > min_req and row['count_sum'] < max_req:
            drive_df = temp_df[(temp_df['lat']==row['lat']) & (temp_df['lng']==row['lng'])]
            drive_df['drive_no'] = drive_no
            drive_no += 1
            fullfilled_drives = pd.concat([fullfilled_drives, drive_df])
            temp_df = temp_df[(temp_df['lat'] != row['lat']) | (temp_df['lng'] != row['lng'])]
        print(drive_no)
    df, drive_no = make_full_filled_clusters(temp_df, drive_no, min_req, max_req)
    fullfilled_drives = pd.concat([fullfilled_drives, df])
# display(fullfilled_drives)
fullfilled_drives.to_csv(output_file_path)


0
0
10
10
10
10


/home/jasmeet16-jtg/projects/virtualenvs/calyx_ds3/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


20
20
30
30
30


/home/jasmeet16-jtg/projects/virtualenvs/calyx_ds3/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/jasmeet16-jtg/projects/virtualenvs/calyx_ds3/lib/python3.5/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


31
31
31
40
50
50
60
60
71
71
80
90
100
110


In [62]:
input_file = os.path.join(BASEPATH, 'pool_recommendation_data/test.csv')
output_file = os.path.join(BASEPATH, 'pool_recommendation_data/final_output_test.csv')
df = pd.read_csv(input_file)

def getuniq(l):
    a = []
    i = 0
    while i < len(l):
        s = l[i]
        a.append(s)
        while i < len(l) and l[i] == s:
            i += 1
    return a


centroid_lat, centroid_lng = [], []
print(getuniq(df['drive_no']))
for i in getuniq(df['drive_no']):
    df1 = df[df['drive_no']==i]
    lt = [df1['lat'].min(), df1['lng'].min()]
    rb = [df1['lat'].max(), df1['lng'].max()]
    centroid_lat.extend([(lt[0] + rb[0])/2] * len(df1))
    centroid_lng.extend([(lt[1] + rb[1])/2] * len(df1))
df['centriod_lat'] = centroid_lat
df['centriod_lng'] = centroid_lng

company_lat, company_lng = 28.503129, 77.083702
df['lat_diff'] = df[['centriod_lat']].apply(lambda x: np.power(abs(x-company_lat), 2))
df['lng_diff'] = df[['centriod_lng']].apply(lambda x: np.power(abs(x-company_lng), 2))
df['distance'] = np.sqrt(df['lat_diff']+df['lng_diff'])
df = df.sort_values(by='distance', ascending=True)
df.to_csv(output_file)

[0, 10, 20, 30, 31, 40, 50, 60, 70, 80, 90, 100, 110]
